In [1]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import stanza

In [2]:
import pickle

In [3]:
import re

In [4]:
import pandas as pd

In [15]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')

2023-01-06 18:52:03 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-01-06 18:52:05 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2023-01-06 18:52:05 INFO: Use device: cpu
2023-01-06 18:52:05 INFO: Loading: tokenize
2023-01-06 18:52:05 INFO: Loading: ner
2023-01-06 18:52:06 INFO: Done loading processors!


In [ ]:
mail_address = []
phone_number = []
name = []

In [8]:
def get_pseudo(category, value):
    """
    get the pseudo value for the requested kind
    :param category: kind of value that gets replaced
    :param value: string of value to replace
    :return: new fake String
    """
    if re.match(r"\[name_\d+\]", value):
        return value
    if re.match(r"\[e-mail_\d+\]", value):
        return value
    if re.match(r"\[phoneNumber_\d+\]", value):
        return value
    if category == "mail":
        try:
            value_index = mail_address.index(value)
            return "[e-mail_"+str(value_index)+"]"
        except:
            mail_address.append(value)
            value_index = mail_address.index(value)
            return "[e-mail_"+str(value_index)+"]"
    if category == "phone_number":
        try:
            value_index = phone_number.index(value)
            return "[phoneNumber_"+str(value_index)+"]"
        except:
            phone_number.append(value)
            value_index = phone_number.index(value)
            return "[phoneNumber_"+str(value_index)+"]"
    if category == "name":
        try:
            value_index = name.index(value)
            return "[name_"+str(value_index)+"]"
        except:
            name.append(value)
            value_index = name.index(value)
            return "[name_"+str(value_index)+"]"

In [9]:
def pseudonymisiere_string(string):
    # finde Telephonenummern und ersetze diese
    match_p = re.findall(
        r'[\+\d]?(\d{2, 3}[-\.\s]??\d{2, 3}[-\.\s]??\d{4} | \(\d{3}\)\s * \d{3}[-\.\s]??\d{4} | \d{3}[-\.\s]??\d{4})',
        string)
    for m in match_p:
        string = string.replace(m, get_pseudo("phone_number", m))

    # finde email adressen
    match_a = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', string)
    for m in match_a:
        string = string.replace(m, get_pseudo("mail", m))

    # finde namen und ersetze diese
    doc = nlp(string)
    i = 0
    while i < len(doc.ents):
        if doc.ents[i].text in string:
            if doc.ents[i].type == 'PERSON' and len(doc.ents[i].text) > 3:
                replace_string = get_pseudo("name", doc.ents[i].text)
                string = string.replace(doc.ents[i].text, replace_string)
        i+=1
    return string

In [ ]:
#data = pd.read_excel("/content/drive/MyDrive/Anonymization/input_anonomization_only_english.xlsx")
data = pd.read_excel("input_anonomization_only_english.xlsx")
field_to_replace = [
    ("work_notes", "complete", []),
    ("opened_by", "name", []),
    #("caller_id","name", []),
    ("assigned_to", "name", []),
    ("u_resolver", "name", []),
    ("comments", "complete", []),
    ("close_notes", "complete", []),
    ("u_assignee_history", "name_comma_seperated", [])
]

In [ ]:
# check all name fields and replace
for field in field_to_replace:
    field[2].clear()
    if field[1]=='name' or field[1]=='name_comma_seperated':
        for index, row in data.iterrows():
            if index % 500 == 0:
                print(field[0]+" "+str(index))
            if (pd.isna(row[field[0]])):
                field[2].append("")
            else:
                if field[1] == "name":
                    field[2].append(get_pseudo("name", row[field[0]]))
                elif field[1] == "name_comma_seperated":
                    all_name = row[field[0]].split(",")
                    new_names = ""
                    for fname in all_name:
                        new_names += " " + get_pseudo("name", fname) + ","
                    new_names = new_names[:-1]
                    new_names = new_names[1:]
                    field[2].append(new_names)

        data[field[0]] = field[2]

    pickle_files = [data,mail_address, phone_number, name, field_to_replace]

    with open('pickle_'+str(field[0])+'_STEP1_names_ONLY.pkl', 'wb') as f:
    #with open('/content/drive/MyDrive/Anonymization/pickle_'+str(field[0])+'_STEP1_names_ONLY.pkl', 'wb') as f:
        pickle.dump(pickle_files, f)

opened_by 0
opened_by 500
opened_by 1000
opened_by 1500
opened_by 2000
opened_by 2500
opened_by 3000
opened_by 3500
opened_by 4000
opened_by 4500
opened_by 5000
opened_by 5500
opened_by 6000
opened_by 6500
opened_by 7000
opened_by 7500
opened_by 8000
opened_by 8500
opened_by 9000
opened_by 9500
opened_by 10000
opened_by 10500
opened_by 11000
opened_by 11500
opened_by 12000
opened_by 12500
opened_by 13000
opened_by 13500
opened_by 14000
opened_by 14500
opened_by 15000
opened_by 15500
opened_by 16000
opened_by 16500
opened_by 17000
opened_by 17500
opened_by 18000
opened_by 18500
opened_by 19000
opened_by 19500
opened_by 20000
opened_by 20500
opened_by 21000
opened_by 21500
opened_by 22000
opened_by 22500
opened_by 23000
opened_by 23500
opened_by 24000
opened_by 24500
opened_by 25000
opened_by 25500
opened_by 26000
opened_by 26500
opened_by 27000
opened_by 27500
opened_by 28000
opened_by 28500
opened_by 29000
opened_by 29500
opened_by 30000
opened_by 30500
opened_by 31000
opened_by 31500


In [ ]:
with open('pickle_u_assignee_history_STEP1_names_ONLY.pkl', 'rb') as f:
    data,mail_address, phone_number, name, field_to_replace = pickle.load(f)

In [ ]:
# replace alle found names in strings
for field in field_to_replace:
    if field[1]=='complete':
        for index, row in data.iterrows():
            if index % 500 == 0:
                print(field[0]+" "+str(index))
            if (pd.isna(row[field[0]])):
                field[2].append("")
            else:
                usedString = row[field[0]]
                for n in name:
                    value_index = name.index(n)
                    usedString = usedString.replace(' '+n+' ', ' '+"[name_"+str(value_index)+"]"+' ')
                field[2].append(usedString)

        data[field[0]] = field[2]
        pickle_files = [data,mail_address, phone_number, name, field_to_replace]

        with open('pickle_'+str(field[0])+'_STEP2_names_ONLY.pkl', 'wb') as f:
            pickle.dump(pickle_files, f)

work_notes 0
work_notes 500
work_notes 1000
work_notes 1500
work_notes 2000
work_notes 2500
work_notes 3000
work_notes 3500
work_notes 4000
work_notes 4500
work_notes 5000
work_notes 5500
work_notes 6000
work_notes 6500
work_notes 7000
work_notes 7500
work_notes 8000
work_notes 8500
work_notes 9000
work_notes 9500
work_notes 10000
work_notes 10500
work_notes 11000
work_notes 11500
work_notes 12000
work_notes 12500
work_notes 13000
work_notes 13500
work_notes 14000
work_notes 14500
work_notes 15000
work_notes 15500
work_notes 16000
work_notes 16500
work_notes 17000
work_notes 17500
work_notes 18000
work_notes 18500
work_notes 19000
work_notes 19500
work_notes 20000
work_notes 20500
work_notes 21000
work_notes 21500
work_notes 22000
work_notes 22500
work_notes 23000
work_notes 23500
work_notes 24000
work_notes 24500
work_notes 25000
work_notes 25500
work_notes 26000
work_notes 26500
work_notes 27000
work_notes 27500
work_notes 28000
work_notes 28500
work_notes 29000
work_notes 29500
work

In [ ]:
field_to_replace[0][2][1]

'2021-01-01 03:24:54 - [name_1] (Work notes)\nWho called : Junwei Ooi_x000D_\nExt Number :   +6046326260_x000D_\nLocations : BMI MY _x000D_\nShort Description :  Siti Ruzaina Abdul Rahman (abdusi29) cannot login to SAP because her employment contact was set as 2020-12-31._x000D_\n_x000D_\nIssue(s) : The user called and mentioned that this user has already extended the work contact until 2021-12-31. He also confirmed that it is better to put the new due date as 2021-12031 because he cannot know what will happen._x000D_\n_x000D_\nMy action taken : I extend the SAP to be 2021-12-31\n\n'

In [12]:
!pip install --upgrade pandas==1.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.4
    Uninstalling pandas-1.3.4:
      Successfully uninstalled pandas-1.3.4


In [14]:
import pandas as pd

In [10]:
print(pd.__version__)


1.4.2


In [11]:
with open('/content/drive/MyDrive/Anonymization/pickle_close_notes_STEP2_names_ONLY.pkl', 'rb') as f:
    data,mail_address, phone_number, name, field_to_replace = pickle.load(f)

In [12]:
# check for names in strings

for field in field_to_replace:
    i=0

    if field[1]=='complete':
        new_field2 = []
        for string in field[2]:
            if i % 5000 == 0:
              print(field[0]+" "+str(i))
            i+=1
            string= string.replace('_x000D_', ' ')
            new_field2.append(pseudonymisiere_string(string))
        field[2].clear()
        for a in new_field2:
          field[2].append(a)
        data[field[0]] = field[2]

    pickle_files = [data,mail_address, phone_number, name, field_to_replace]

    with open('/content/drive/MyDrive/Anonymization/pickle_'+str(field[0])+'_STEP3_names_ONLY.pkl', 'wb') as f:
        pickle.dump(pickle_files, f)

work_notes 0
work_notes 5000
work_notes 10000
work_notes 15000
work_notes 20000
work_notes 25000
work_notes 30000
work_notes 35000
work_notes 40000
work_notes 45000
work_notes 50000
work_notes 55000
work_notes 60000
work_notes 65000
comments 0
comments 5000
comments 10000
comments 15000
comments 20000
comments 25000
comments 30000
comments 35000
comments 40000
comments 45000
comments 50000
comments 55000
comments 60000
comments 65000
close_notes 0
close_notes 5000
close_notes 10000
close_notes 15000
close_notes 20000
close_notes 25000
close_notes 30000
close_notes 35000
close_notes 40000
close_notes 45000
close_notes 50000
close_notes 55000
close_notes 60000
close_notes 65000


In [15]:
print(field_to_replace[0][2][4])

2021-01-01 04:55:38 - [name_1] (Work notes)
Who called : [name_7813] Tan 
Ext Number :   
Call time :   (UTC+8) 
Computer Hostname :  MY01-C1155 
Locations : BMI MY 
Short Description :  the user cannot connect to VPN. She said that the connection failed. 
 
My action taken : I checked her PC version which is 1809. I asked her to restart the PC and reconnect again.




In [23]:
with open('pickle_close_notes_STEP3_names_ONLY.pkl', 'rb') as f:
    data,mail_address, phone_number, name, field_to_replace = pickle.load(f)

In [24]:
print(len(name))

32391


In [26]:
# replace names in strings

for field in field_to_replace:
    i=0

    if field[1]=='complete':
        completeString = ""

        for string in field[2]:
            if i % 5000 == 0:
              print(field[0]+" "+str(i))
            i+=1
            completeString+= string.replace('_x000D_', ' ')+' IAMDIVIDERSPLITTERALLINONE '

        i=0
        for n in name:
            if i % 2500 == 0:
                print(field[0]+" "+str(i))
            value_index = name.index(n)
            completeString = completeString.replace(' '+n+' ', ' '+"[name_"+str(value_index)+"]"+' ')
            i+=1
        print('split_start')
        new_field2 = completeString.split(' IAMDIVIDERSPLITTERALLINONE ')
        new_field2.pop()
        print('split_end')
        field[2].clear()
        for a in new_field2:
          field[2].append(a)

        data[field[0]] = field[2]
        pickle_files = [data,mail_address, phone_number, name, field_to_replace]

        with open('pickle_'+str(field[0])+'_STEP4_names_ONLY.pkl', 'wb') as f:
            pickle.dump(pickle_files, f)


work_notes 0
work_notes 5000
work_notes 10000
work_notes 15000
work_notes 20000
work_notes 25000
work_notes 30000
work_notes 35000
work_notes 40000
work_notes 45000
work_notes 50000
work_notes 55000
work_notes 60000
work_notes 65000
work_notes 0
work_notes 2500
work_notes 5000
work_notes 7500
work_notes 10000
work_notes 12500
work_notes 15000
work_notes 17500
work_notes 20000
work_notes 22500
work_notes 25000
work_notes 27500
work_notes 30000
split_start
split_end
comments 0
comments 5000
comments 10000
comments 15000
comments 20000
comments 25000
comments 30000
comments 35000
comments 40000
comments 45000
comments 50000
comments 55000
comments 60000
comments 65000
comments 0
comments 2500
comments 5000
comments 7500
comments 10000
comments 12500
comments 15000
comments 17500
comments 20000
comments 22500
comments 25000
comments 27500
comments 30000
split_start
split_end
close_notes 0
close_notes 5000
close_notes 10000
close_notes 15000
close_notes 20000
close_notes 25000
close_notes 3

In [18]:

new_field2 = completeString.split(' IAMDIVIDERSPLITTERALLINONE ')

In [19]:
len(new_field2)


67172

In [20]:
new_field2.pop()

''

In [22]:
len(new_field2)

67171

In [ ]:
import pickle

In [27]:
data.to_excel("output_anonomization_no_names_only_english_version2.xlsx")

C:\Users\Cornelius\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'http://212.0.40.137/ALSWebImport/

2021-03-11%2014:21:34%20-%20Noah%20Olawale%20(Additional%20comments)
A%20Teams%20invitation%20has%20been%20sent%20to%20have%20a%20better%20view%20on%20what%20the%20issue%20might%20be.

2021-03-04%2001:49:02%20-%20Felix%20Geng%20(Additional%20comments)
Dear%20network%20&%20security%20team_x000D_
is%20there%20progress%20update%20for%20this%20ticket.

2021-02-08%2007:46:16%20-%20Noah%20Olawale%20(Additional%20comments)
%5bcode%5d%3cb%3eAttachment%202021-02-08%2007_45_39-de08-wtwi02%20-%20Remote%20Desktop%20Connection.png%20was%20added%20on%202021-02-08%2006:46:07%20by%20olawnopl.%3c/b%3e%5b/code%5d

2021-02-08%2007:46:16%20-%20Noah%20Olawale%20(Additional%20comments)
%5bcode%5d%3cb%3eAttachment%202021-02-08%2007_45_39-de08-wtwi02%20-%20Remote%20Desktop%20Connection.png%20was%20added%20on%202021-02-08%2006:46:07%20by%20olawnopl.%3c/b%3e%5b/code%5d

2021

In [28]:
# initialize data of lists.
na = {'ValueName': name}
ph = {'ValuePhone': phone_number}
ma = {'ValueMail': mail_address}
# Create DataFrame
df_na = pd.DataFrame(na)
df_ph = pd.DataFrame(ph)
df_ma = pd.DataFrame(ma)
df_na.to_excel("pseudo_key_name_no_names_v1.xlsx",encoding='utf-8')
df_ph.to_excel("pseudo_key_phone_no_names_v1.xlsx",encoding='utf-8')
df_ma.to_excel("pseudo_key_mail_no_names_v1.xlsx",encoding='utf-8')